# 랜덤 포레스트

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestClassifier
from pandas.core.common import random_state
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

## 데이터 불러오기

In [3]:
# Disease 

df = pd.read_csv('/content/drive/MyDrive/health/data/data_over_sampling(smote).csv', encoding='cp949') # 원본 그대로 오버샘플링(1:3:1) : 스모트
df

,SEX,BTH_G,SBP,DBP,FBS,BMI,DIS,Dis
0,0,1,116,78,94,16.600000,4,0
1,0,1,100,60,79,22.300000,4,0
2,0,1,100,60,87,21.900000,4,0
3,0,1,111,70,72,20.200000,4,0
4,0,1,120,80,98,20.000000,4,0
...,...,...,...,...,...,...,...,...
1481319,0,27,119,64,94,21.927633,1,1
1481320,0,27,145,70,108,25.343427,1,1
1481321,1,16,123,72,104,20.760203,2,1
1481322,0,27,115,75,143,21.647994,1,1


In [4]:
df.drop(df[df['Dis']==0].index, inplace=True)
df

,SEX,BTH_G,SBP,DBP,FBS,BMI,DIS,Dis
52,0,1,90,50,86,20.100000,2,1
97,0,1,120,80,90,27.000000,3,1
133,0,1,100,60,112,31.800000,2,1
204,0,1,170,120,86,23.300000,2,1
255,0,1,135,80,104,35.000000,2,1
...,...,...,...,...,...,...,...,...
1481319,0,27,119,64,94,21.927633,1,1
1481320,0,27,145,70,108,25.343427,1,1
1481321,1,16,123,72,104,20.760203,2,1
1481322,0,27,115,75,143,21.647994,1,1


In [5]:
# 1,0 정답 레이블 삭제
df.drop(['Dis'], axis=1, inplace=True)
df

,SEX,BTH_G,SBP,DBP,FBS,BMI,DIS
52,0,1,90,50,86,20.100000,2
97,0,1,120,80,90,27.000000,3
133,0,1,100,60,112,31.800000,2
204,0,1,170,120,86,23.300000,2
255,0,1,135,80,104,35.000000,2
...,...,...,...,...,...,...,...
1481319,0,27,119,64,94,21.927633,1
1481320,0,27,145,70,108,25.343427,1
1481321,1,16,123,72,104,20.760203,2
1481322,0,27,115,75,143,21.647994,1


In [6]:
df.notnull().sum()

SEX      740662
BTH_G    740662
SBP      740662
DBP      740662
FBS      740662
BMI      740662
DIS      740662
dtype: int64

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 740662 entries, 52 to 1481323
Data columns (total 7 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   SEX     740662 non-null  int64  
 1   BTH_G   740662 non-null  int64  
 2   SBP     740662 non-null  int64  
 3   DBP     740662 non-null  int64  
 4   FBS     740662 non-null  int64  
 5   BMI     740662 non-null  float64
 6   DIS     740662 non-null  int64  
dtypes: float64(1), int64(6)
memory usage: 45.2 MB


In [8]:
# feature와 target 분리
y = np.array(df['DIS'])
x = df.drop(['DIS'], axis=1)
x_train=np.array(x)
print(x,y)

# 데이터 정규화
scaler=MinMaxScaler()
x_train=scaler.fit_transform(x_train)

# train test 데이터 split
# test_size=0.2, random_state=42
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.4, shuffle=True)

         SEX  BTH_G  SBP  DBP  FBS        BMI
52         0      1   90   50   86  20.100000
97         0      1  120   80   90  27.000000
133        0      1  100   60  112  31.800000
204        0      1  170  120   86  23.300000
255        0      1  135   80  104  35.000000
...      ...    ...  ...  ...  ...        ...
1481319    0     27  119   64   94  21.927633
1481320    0     27  145   70  108  25.343427
1481321    1     16  123   72  104  20.760203
1481322    0     27  115   75  143  21.647994
1481323    0     15  140   80  101  24.281789

[740662 rows x 6 columns] [2 3 2 ... 2 1 2]


In [9]:
# df를 train/validation/test set으로 나누기
# 전체 데이터에서 학습데이터셋과 테스트셋을 나눔
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.4, shuffle=True)

# train set에서 train과 validation set를 나눔
x_test,x_val,y_test,y_val=train_test_split(x_test,y_test,test_size=0.5, shuffle=True)


print(x_train.shape)
print(y_train.shape)
print(x_val.shape)
print(y_val.shape)
print(x_test.shape)
print(y_test.shape)

(444397, 6)
(444397,)
(148133, 6)
(148133,)
(148132, 6)
(148132,)


In [10]:
# 랜덤포레스트 without tuning
rf = RandomForestClassifier(random_state=42)
rf.fit(x_train, y_train)
forest_predictions = rf.predict(x_test)

accuracy_score(y_test, forest_predictions)
forest_predictions = rf.predict(x_test)

In [11]:
# 데이터 100만 개 사용
print(confusion_matrix(y_test, forest_predictions))

[[23649 11225  1530]
 [ 6230 87021  2489]
 [ 2663  5490  7835]]


In [12]:
rf = RandomForestClassifier(n_estimators=100, random_state=42).fit(x_train,y_train)

print("train score : {}".format(rf.score(x_train, y_train)))
print("val score : {}".format(rf.score(x_val, y_val)))

#print("test score : {}".format(dTree.score(x_test,y_test)))
predict_y = rf.predict(x_test)

from sklearn.metrics import classification_report
print(classification_report(y_test, rf.predict(x_test)))
print("test score : {}".format(rf.score(x_test, y_test)))

train score : 0.9984158308899475
val score : 0.7997070200427994
              precision    recall  f1-score   support

           1       0.73      0.65      0.69     36404
           2       0.84      0.91      0.87     95740
           3       0.66      0.49      0.56     15988

    accuracy                           0.80    148132
   macro avg       0.74      0.68      0.71    148132
weighted avg       0.79      0.80      0.79    148132

test score : 0.7999959495585018


# SMOTE + 질병 1, 2, 3 비율 => 1 : 1 : 1

In [13]:
# Disease 

df2 = pd.read_csv('/content/drive/MyDrive/health/data/data(over_dis_vs_nor_smote).csv', encoding='cp949') # 1~3 비율 맞춰서 오버샘플링 : 스모트
df2

,SEX,BTH_G,SBP,DBP,FBS,BMI,DIS,Dis
0,0,1,90,50,86,20.0,2,1
1,0,1,120,80,90,27.0,3,1
2,0,1,100,60,112,32.0,2,1
3,0,1,170,120,86,23.0,2,1
4,0,1,135,80,104,35.0,2,1
...,...,...,...,...,...,...,...,...
1481319,0,22,150,90,95,28.0,1,1
1481320,0,14,138,75,178,23.0,1,1
1481321,0,25,118,74,143,21.0,1,1
1481322,1,20,128,79,87,23.0,1,1


In [14]:
df2.drop(df2[df2['Dis']==0].index, inplace=True)
df2

,SEX,BTH_G,SBP,DBP,FBS,BMI,DIS,Dis
0,0,1,90,50,86,20.0,2,1
1,0,1,120,80,90,27.0,3,1
2,0,1,100,60,112,32.0,2,1
3,0,1,170,120,86,23.0,2,1
4,0,1,135,80,104,35.0,2,1
...,...,...,...,...,...,...,...,...
1481319,0,22,150,90,95,28.0,1,1
1481320,0,14,138,75,178,23.0,1,1
1481321,0,25,118,74,143,21.0,1,1
1481322,1,20,128,79,87,23.0,1,1


In [15]:
# 1,0 정답 레이블 삭제
df2.drop(['Dis'], axis=1, inplace=True)
df2

,SEX,BTH_G,SBP,DBP,FBS,BMI,DIS
0,0,1,90,50,86,20.0,2
1,0,1,120,80,90,27.0,3
2,0,1,100,60,112,32.0,2
3,0,1,170,120,86,23.0,2
4,0,1,135,80,104,35.0,2
...,...,...,...,...,...,...,...
1481319,0,22,150,90,95,28.0,1
1481320,0,14,138,75,178,23.0,1
1481321,0,25,118,74,143,21.0,1
1481322,1,20,128,79,87,23.0,1


In [16]:
df2.notnull().sum()

SEX      740662
BTH_G    740662
SBP      740662
DBP      740662
FBS      740662
BMI      740662
DIS      740662
dtype: int64

In [17]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 740662 entries, 0 to 1481323
Data columns (total 7 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   SEX     740662 non-null  int64  
 1   BTH_G   740662 non-null  int64  
 2   SBP     740662 non-null  int64  
 3   DBP     740662 non-null  int64  
 4   FBS     740662 non-null  int64  
 5   BMI     740662 non-null  float64
 6   DIS     740662 non-null  int64  
dtypes: float64(1), int64(6)
memory usage: 45.2 MB


In [18]:
# feature와 target 분리
y = np.array(df2['DIS'])
x = df2.drop(['DIS'], axis=1)
x_train=np.array(x)
print(x,y)

# 데이터 정규화
scaler=MinMaxScaler()
x_train=scaler.fit_transform(x_train)

# train test 데이터 split
# test_size=0.2, random_state=42
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.4, shuffle=True)

         SEX  BTH_G  SBP  DBP  FBS   BMI
0          0      1   90   50   86  20.0
1          0      1  120   80   90  27.0
2          0      1  100   60  112  32.0
3          0      1  170  120   86  23.0
4          0      1  135   80  104  35.0
...      ...    ...  ...  ...  ...   ...
1481319    0     22  150   90   95  28.0
1481320    0     14  138   75  178  23.0
1481321    0     25  118   74  143  21.0
1481322    1     20  128   79   87  23.0
1481323    0     18  130   80  195  24.0

[740662 rows x 6 columns] [2 3 2 ... 1 1 3]


In [19]:
# df를 train/validation/test set으로 나누기
# 전체 데이터에서 학습데이터셋과 테스트셋을 나눔
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.4, shuffle=True)

# train set에서 train과 validation set를 나눔
x_test,x_val,y_test,y_val=train_test_split(x_test,y_test,test_size=0.5, shuffle=True)

print(x_train.shape)
print(y_train.shape)
print(x_val.shape)
print(y_val.shape)
print(x_test.shape)
print(y_test.shape)

(444397, 6)
(444397,)
(148133, 6)
(148133,)
(148132, 6)
(148132,)


In [20]:
# 랜덤포레스트 without tuning
rf = RandomForestClassifier(random_state=42)
rf.fit(x_train, y_train)
forest_predictions = rf.predict(x_test)

accuracy_score(y_test, forest_predictions)
forest_predictions = rf.predict(x_test)

In [21]:
# 데이터 100만 개 사용
print(confusion_matrix(y_test, forest_predictions))

[[42328  5924  3829]
 [ 6517 37309  4793]
 [ 3962  4570 38900]]


In [22]:
rf = RandomForestClassifier(n_estimators=100, random_state=42).fit(x_train,y_train)

print("train score : {}".format(rf.score(x_train, y_train)))
print("val score : {}".format(rf.score(x_val, y_val)))

#print("test score : {}".format(dTree.score(x_test,y_test)))
predict_y = rf.predict(x_test)

from sklearn.metrics import classification_report
print(classification_report(y_test, rf.predict(x_test)))
print("test score : {}".format(rf.score(x_test, y_test)))

train score : 0.9739309671307412
val score : 0.7981881147347316
              precision    recall  f1-score   support

           1       0.80      0.81      0.81     52081
           2       0.78      0.77      0.77     48619
           3       0.82      0.82      0.82     47432

    accuracy                           0.80    148132
   macro avg       0.80      0.80      0.80    148132
weighted avg       0.80      0.80      0.80    148132

test score : 0.8002119731050684


---

In [23]:
dTree = DecisionTreeClassifier(max_depth=10,random_state=42).fit(x_train,y_train)

print("train score : {}".format(dTree.score(x_train, y_train)))
#print("val score : {}".format(dTree.score(x_val, y_val)))

#print("test score : {}".format(dTree.score(x_test,y_test)))
predict_y = rf.predict(x_test)

from sklearn.metrics import classification_report
print(classification_report(y_test, rf.predict(x_test)))
print("test score : {}".format(rf.score(x_test, y_test)))

train score : 0.5893176596601687
              precision    recall  f1-score   support

           1       0.80      0.81      0.81     52081
           2       0.78      0.77      0.77     48619
           3       0.82      0.82      0.82     47432

    accuracy                           0.80    148132
   macro avg       0.80      0.80      0.80    148132
weighted avg       0.80      0.80      0.80    148132

test score : 0.8002119731050684


In [25]:
import graphviz
from sklearn.tree import export_graphviz

export_graphviz(dTree, out_file='tree.dot', class_names= ['1','2','3'], feature_names=df.columns[:-1], impurity=False, filled=True)
with open('tree.dot') as file_reader:
  dot_graph = file_reader.read()

dot = graphviz.Source(dot_graph)
dot.render(filename='tree.png')

'tree.png.pdf'

In [26]:
with open("tree.dot") as f:
  dot_graph = f.read()
graphviz.Source(dot_graph)